In [ ]:
from preprocessing.pre_process_utils import CDCPArgumentationDoc
from utils.data_processors import ArgMinRelClassProcessorCDCPRBERTJointLearning, ArgMinRelClassProcessorCDCPWithDistance
from transformers import BertTokenizer
from utils.metrics import compute_metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data/cdcp/original")

# doc_ids = [file.split('.')[0] for file in os.listdir(data_dir) if file.split(".")[-1] == "txt"]
# docs = [CDCPArgumentationDoc(data_dir + '/' + doc_id) for doc_id in doc_ids]

In [ ]:
rbert_output_dir = os.path.join(os.getcwd(), "output/bert-rbert-jl-2+None+cdcp_relclass_rbert_jl+512+LR2e-5+WD0+FZ0+EP20+BS8")

tokenizer = BertTokenizer.from_pretrained(
    rbert_output_dir,
    do_lower_case=True,
    cache_dir=None,
)
tokenizer.add_special_tokens({"additional_special_tokens": ['<src>', '</src>', '<trg>', '</trg>']})

dp = ArgMinRelClassProcessorCDCPRBERTJointLearning()
docs = dp.read_cdcp_docs(os.path.join(data_dir, 'test'))

longest_example = 0
counts = {'reason': 0,
          'evidence': 0,
          'noRel': 0,
          'policy': 0, 'fact': 0, 'testimony': 0, 'value': 0, 'reference': 0}

text_info = []
rows = []
skipped_idxs = []

count = 0
for doc_id, doc in enumerate(docs):
    for c_src_idx, c_src in enumerate(doc.prop_offsets):
        for c_trg_idx, c_trg in enumerate(doc.prop_offsets):
            if c_src_idx == c_trg_idx:
                continue
            count += 1
            if (c_src_idx, c_trg_idx) in doc.links_lists['locate']:
                rel_type = doc.links_lists['link'][doc.links_lists['locate'].index((c_src_idx, c_trg_idx))]
            else:
                rel_type = 'noRel'
            c_text_order = [['src', c_src], ['trg', c_trg]] if c_src_idx < c_trg_idx else [['trg', c_trg], ['src', c_src]]
            text_a = dp.create_text_with_tags(doc.raw_text, c_text_order)
            tokens = tokenizer.tokenize(text_a)
            if len(tokens) > 512:
                skipped_idxs.append(count)
                continue
            for prop_idx, prop in enumerate(doc.prop_offsets):
                counts[doc.prop_labels[prop_idx]] += 1
            counts[rel_type] += 1

            source_first = c_src_idx < c_trg_idx
            relation_label = dp.get_labels().index('__label__' + rel_type)
            source_label = dp.get_component_labels().index(doc.prop_labels[c_src_idx])
            target_label = dp.get_component_labels().index(doc.prop_labels[c_trg_idx])
            source_len = tokens.index(dp.TAGS['src'][1].strip()) - tokens.index(dp.TAGS['src'][0].strip())
            target_len = tokens.index(dp.TAGS['trg'][1].strip()) - tokens.index(dp.TAGS['trg'][0].strip())
            distance = abs(c_trg_idx - c_src_idx)
            doc_len = len(tokens)
            bridge_len = tokens.index(dp.TAGS['trg'][0].strip()) - tokens.index(dp.TAGS['src'][1].strip()) - 1 \
                if source_first else tokens.index(dp.TAGS['src'][0].strip()) - tokens.index(dp.TAGS['trg'][1].strip()) - 1
            non_component_token_len = doc_len - source_len - target_len
            outer_len = non_component_token_len - bridge_len
            non_component_percentage = non_component_token_len / doc_len * 100

            row_dict = {'source_first': source_first,
                        'relation_label': relation_label,
                        'source_label': source_label,
                        'target_label': target_label,
                        'source_len': source_len,
                        'target_len': target_len,
                        'Component Distance': distance,
                        'Document Length': doc_len,
                        'Bridge Length': bridge_len,
                        'Non-component Length': non_component_token_len,
                        'Non-component %': non_component_percentage,
                        'Outer Length': outer_len,
                        'document_id': doc_id}

            text_entry = {'text': text_a,
                          'tokens': tokens,
                          'document_id': doc_id,
                          'relation_label': relation_label}
            rows.append(row_dict)
            text_info.append(text_entry)

examples_df = pd.DataFrame(rows)

In [ ]:
rbert_relation_preds = []
rbert_source_preds = []
rbert_target_preds = []

output_eval_file = os.path.join(rbert_output_dir, "eval_relation_preds.txt")
with open(output_eval_file, "r") as reader:
    for p in reader:
        for p_label in list(p):
            rbert_relation_preds.append(int(p_label))

output_eval_file = os.path.join(rbert_output_dir, "eval_source_preds.txt")
with open(output_eval_file, "r") as reader:
    for p in reader:
        for p_label in list(p):
            rbert_source_preds.append(int(p_label))

output_eval_file = os.path.join(rbert_output_dir, "eval_target_preds.txt")
with open(output_eval_file, "r") as reader:
    for p in reader:
        for p_label in list(p):
            rbert_target_preds.append(int(p_label))

df = examples_df.copy()

df['rbert_relation_pred'] = np.array(rbert_relation_preds)
df['rbert_source_pred'] = np.array(rbert_source_preds)
df['rbert_target_pred'] = np.array(rbert_target_preds)

In [ ]:
dist_output_dir = os.path.join(os.getcwd(), "output/bert-distance+2FC-RelU+cdcp_relclass_distance+166+LR1e-5+WD0+FZ0+EP15+BS20")

dist_relation_preds = []

output_eval_file = os.path.join(dist_output_dir, "eval_relation_preds.txt")
with open(output_eval_file, "r") as reader:
    for p in reader:
        for idx, p_label in enumerate(list(p)):
            if idx in skipped_idxs:
                continue
            dist_relation_preds.append(int(p_label))

df['dist_relation_pred'] = np.array(dist_relation_preds)

In [ ]:
rbert_metrics = compute_metrics('cdcp_relclass_resnet_jl', df['rbert_relation_pred'], df['relation_label'])
dist_metrics = compute_metrics('cdcp_relclass_resnet_jl', df['dist_relation_pred'], df['relation_label'])

In [ ]:
plt.hist(df['Document Length'], bins=15)

In [ ]:
rsn = df.loc[df['relation_label'] == 1]

rbert_rsn_tp = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)]
rbert_rsn_tn = df[(df['relation_label'] != 1) & (df['rbert_relation_pred'] != 1)]
rbert_rsn_fp = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] != 1)]
rbert_rsn_fn = df[(df['relation_label'] != 1) & (df['rbert_relation_pred'] == 1)]
rbert_rsn_fp_fn = df[((df['relation_label'] == 1) & (df['rbert_relation_pred'] != 1)) | ((df['relation_label'] != 1) & (df['rbert_relation_pred'] == 1))]
rbert_rsn_tp_tn = df[((df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)) | ((df['relation_label'] != 1) & (df['rbert_relation_pred'] != 1))]

In [ ]:
cm = confusion_matrix(df['relation_label'], df['rbert_relation_pred'])

disp = ConfusionMatrixDisplay(cm, ['None', 'Reason', 'Evidence'])

disp.plot()

disp.figure_.savefig('images/RBERT-cm')

In [ ]:
cm2 = confusion_matrix(df['relation_label'], df['dist_relation_pred'])

disp2 = ConfusionMatrixDisplay(cm2, ['None', 'Reason', 'Evidence'])

disp2.plot()

disp2.figure_.savefig('images/DIST-cm')

In [ ]:
rbert_rsn_c = rsn[rsn['rbert_relation_pred'] == 1]
rbert_rsn_f = rsn[rsn['rbert_relation_pred'] != 1]

In [ ]:
df['relation_label'].value_counts()

In [ ]:
def process_means(dframe, name):
    all_comps = np.concatenate([dframe['source_len'].values, dframe['target_len'].values])
    dframe_mean = pd.Series(dframe.mean()[:], name=name)
    dframe_mean['count'] = len(dframe)
    dframe_mean['component_len'] = all_comps.mean()
    dframe_mean['component_len_std'] = all_comps.std()
    to_drop = ['rbert_relation_pred', 'rbert_source_pred', 'rbert_target_pred', 'dist_relation_pred', 'source_label',
               'target_label', 'relation_label', 'document_id']
    dframe_mean = dframe_mean.drop(to_drop)
    to_std = ['Bridge Length', 'Document Length', 'Non-component Length', 'Outer Length', 'Component Distance', 'Non-component %']
    stds = dframe[to_std].std()
    stds = stds.rename({to_std[i]: to_std[i] + " Std" for i in range(6)})
    dframe_mean = dframe_mean.append(stds)

    # vcs = dframe['source_label'].value_counts(sort=False).reindex(df.source_label.unique(), fill_value=0)
    # vcs = vcs / dframe_mean['count']
    # vcs = vcs.rename({i: dp.get_component_labels()[i] for i in range(5)})
    # dframe_mean.append(vcs).rename(name)
    order = ['source_first', 'source_len', 'target_len', 'component_len', 'component_len_std', 'Document Length', 'Document Length Std',
             'Bridge Length', 'Bridge Length Std', 'Component Distance', 'Component Distance Std',
             'Non-component Length', 'Non-component Length Std', 'Outer Length', 'Outer Length Std',
             'Non-component %', 'Non-component % Std', 'count']
    dframe_mean = dframe_mean.reindex(index = order)
    return dframe_mean.rename(name)

process_means(rsn, 'All Reason')

In [ ]:
plt.hist(rsn['Bridge Length'], bins=25)

# rsn_mean = pd.Series(rsn.mean()[:], name='All Reason')
# rsn_mean['count'] = len(rsn)
rsn_mean = process_means(rsn, 'All Reason')

In [ ]:
plt.hist(rbert_rsn_c['Bridge Length'], bins=25)

rbert_rsn_c_mean = process_means(rbert_rsn_c, 'Correct Reason RBERT')

In [ ]:
plt.hist(rbert_rsn_f['Bridge Length'], bins=25)

rbert_rsn_f_mean = process_means(rbert_rsn_f, 'False Reason RBERT')

In [ ]:
dist_rsn_c = rsn[rsn['dist_relation_pred'] == 1]
dist_rsn_f = rsn[rsn['dist_relation_pred'] != 1]

In [ ]:
plt.hist(rsn['Bridge Length'], bins=25)

dist_rsn_c_mean = process_means(dist_rsn_c, 'Correct Reason DIST')

In [ ]:
plt.hist(dist_rsn_f['Bridge Length'], bins=25)

dist_rsn_f_mean = process_means(dist_rsn_f, 'False Reason DIST')

In [ ]:
means = pd.DataFrame(rsn_mean)
means = means.join(rbert_rsn_c_mean)
means = means.join(rbert_rsn_f_mean)
means = means.join(dist_rsn_c_mean)
means = means.join(dist_rsn_f_mean)
means = means.round(2)

means

In [ ]:
df

In [ ]:
stat = 'Non-component %'
bins = 15
data1 = df[(df['relation_label'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)]['Component Distance']
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Non-component / doc length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.scatter(data1, data2)

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component Length / Doc length", size=14)
plt.ylabel("Distance", size=14)
# plt.title(title)
plt.legend(loc='upper right')
# plt.savefig('images/RBERT-NCOT')



In [ ]:
stat = 'Bridge Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Bridge length - Reason: Correct predictions against all examples'


plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Bridge Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERT-BL')

In [ ]:
stat = 'Component Distance'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Component Distance - Reason: Correct predictions against all examples'


plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Component Distance", size=14)
plt.ylabel("Frequency", size=14)
plt.xticks([x + 1  for x in range(12)])
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERT-CD')

In [ ]:
stat = 'Non-component Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Non-component length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERT-NCL')

In [ ]:
stat = 'Non-component %'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Proportion of non-component tokens - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component token %", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERT-NCOT')

In [ ]:
stat = 'Non-component %'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1) & (df['Component Distance'] == 1)][stat]
data2 = df[(df['relation_label'] == 1) & (df['Component Distance'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Proportion of non-component tokens - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component token %", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERT-NCOT-D1')

In [ ]:
stat = 'Document Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Document length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Document Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERT-DL')

In [ ]:
stat = 'target_len'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'Outer length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Outer Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERT-OL')

In [ ]:
stat = 'Bridge Length'
bins = 30
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST \n Bridge length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Bridge Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/DIST-BL')

In [ ]:
stat = 'Component Distance'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST \n Component Distance - Reason: Correct predictions against all examples'


plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Component Distance", size=14)
plt.ylabel("Frequency", size=14)
plt.xticks([x + 1  for x in range(12)])
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/DIST-CD')

In [ ]:
stat = 'Non-component Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST \n Non-component length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/DIST-NCL')

In [ ]:
stat = 'Non-component Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1) & (df['Component Distance'] == 1)][stat]
data2 = df[(df['relation_label'] == 1) & (df['Component Distance'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST \n Non-component length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/DIST-NCL-D1')

In [ ]:
stat = 'source_len'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST \n Non-component length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/DIST-SL')

In [ ]:
def plot_proportional_change(stat, verbose=False):
    data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
    data2 = df[(df['relation_label'] == 1)][stat]
    bins = 15

    _, bin1 = pd.cut(data2, bins, retbins=True)
    bins = pd.IntervalIndex.from_breaks(bin1)

    binned_dist = pd.cut(data1, bins)
    bin_vc_dist = binned_dist.value_counts()

    binned_all = pd.cut(data2, bins)
    bin_vc_all = binned_all.value_counts()

    dist_NCP_proportion = (bin_vc_dist / bin_vc_all).fillna(0)

    data3 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]

    binned_rbert = pd.cut(data3, bins)
    bin_vc_rbert = binned_rbert.value_counts()

    rbert_NCP_proportion = (bin_vc_rbert / bin_vc_all).fillna(0)
    proportion_change = (rbert_NCP_proportion - dist_NCP_proportion) * 100

    midpoints = [(bin1[x] + bin1[x+1]) / 2 for x in range(0, len(bin1) -1 , 1)]

    p_plus = [x for x in proportion_change.values if x >= 0]
    p_neg = [x for x in proportion_change.values if x < 0]
    mid_plus = [midpoints[x] for x, y in enumerate(proportion_change.values) if y >= 0]
    mid_neg = [midpoints[x] for x, y in enumerate(proportion_change.values) if y < 0]

    if verbose:
        print(p_plus)
        print(bin1)
        print((bin1[0] + bin1[1]) / 2)
        print(midpoints)
        print(max(proportion_change.index).right)
        print(len(midpoints))
        print(len(proportion_change))
        print(len(p_plus))
        print(len(mid_plus))
        print(len(p_neg))
        print(len(mid_neg))

    # plt.bar(midpoints, proportion_change, alpha=0.7, width=(max(proportion_change.index).right / len(midpoints)))
    plt.bar(mid_plus, p_plus, alpha=0.7, width=(max(proportion_change.index).right / len(midpoints)))
    plt.bar(mid_neg, p_neg, alpha=0.7, color='red', width=(max(proportion_change.index).right / len(midpoints)))

    plt.axhline(y=0, color='black', linestyle='-')

    # plt.title(stat + " proportional change between RBERT and DIST Correct predictions")
    plt.xlabel(stat, size=14)
    plt.ylabel("%", size=14)
    plt.yticks([ x for x in range(-100, 125, 25)])
    plt.show()

In [ ]:
def plot_proportional_change_nobins(stat, verbose=False):
    data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
    data2 = df[(df['relation_label'] == 1)][stat]

    vc_dist = data1.value_counts().sort_index()[0:5]
    vc_all = data2.value_counts().sort_index()[0:5]

    dist_NCP_proportion = (vc_dist / vc_all).fillna(0)

    data3 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]

    vc_rbert = data3.value_counts().sort_index()[0:5]

    rbert_NCP_proportion = (vc_rbert / vc_all).fillna(0)

    change = (rbert_NCP_proportion - dist_NCP_proportion) * 100

    midpoints = [1,2,3,4,5]

    print(change)

    p_plus = [x for x in change.values if x >= 0]
    p_neg = [x for x in change.values if x < 0]
    mid_plus = [midpoints[x] for x, y in enumerate(change.values) if y >= 0]
    mid_neg = [midpoints[x] for x, y in enumerate(change.values) if y < 0]

    if verbose:
        print(p_plus)
        print(midpoints)
        print(max(change.index).right)
        print(len(midpoints))
        print(len(change))
        print(len(p_plus))
        print(len(mid_plus))
        print(len(p_neg))
        print(len(mid_neg))

    # plt.bar(midpoints, proportion_change, alpha=0.7, width=(max(proportion_change.index).right / len(midpoints)))
    plt.bar(mid_plus, p_plus, alpha=0.7, width=1)
    plt.bar(mid_neg, p_neg, alpha=0.7, color='red', width=1)

    plt.axhline(y=0, color='black', linestyle='-')

    # plt.title(stat + " proportional change between RBERT and DIST Correct predictions")
    plt.xlabel(stat, size=14)
    plt.ylabel("%", size=14)
    plt.xticks([1,2,3,4,5])
    plt.yticks([ x for x in range(-100, 125, 25)])
    # plt.savefig('images/CD-Prop')
    plt.show()

plot_proportional_change_nobins('Component Distance')

In [ ]:
def plot_proportional_change_d1(stat, verbose=False):
    data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1) & (df['Component Distance'] == 1)][stat]
    data2 = df[(df['relation_label'] == 1) & (df['Component Distance'] == 1)][stat]
    bins = 15

    _, bin1 = pd.cut(data2, bins, retbins=True)
    bins = pd.IntervalIndex.from_breaks(bin1)

    binned_dist = pd.cut(data1, bins)
    bin_vc_dist = binned_dist.value_counts()

    binned_all = pd.cut(data2, bins)
    bin_vc_all = binned_all.value_counts()

    dist_NCP_proportion = (bin_vc_dist / bin_vc_all).fillna(0)

    data3 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1) & (df['Component Distance'] == 1)][stat]

    binned_rbert = pd.cut(data3, bins)
    bin_vc_rbert = binned_rbert.value_counts()

    rbert_NCP_proportion = (bin_vc_rbert / bin_vc_all).fillna(0)
    proportion_change = (rbert_NCP_proportion - dist_NCP_proportion) * 100

    midpoints = [(bin1[x] + bin1[x+1]) / 2 for x in range(0, len(bin1) -1 , 1)]

    p_plus = [x for x in proportion_change.values if x >= 0]
    p_neg = [x for x in proportion_change.values if x < 0]
    mid_plus = [midpoints[x] for x, y in enumerate(proportion_change.values) if y >= 0]
    mid_neg = [midpoints[x] for x, y in enumerate(proportion_change.values) if y < 0]

    if verbose:
        print(p_plus)
        print(bin1)
        print((bin1[0] + bin1[1]) / 2)
        print(midpoints)
        print(max(proportion_change.index).right)
        print(len(midpoints))
        print(len(proportion_change))
        print(len(p_plus))
        print(len(mid_plus))
        print(len(p_neg))
        print(len(mid_neg))

    # plt.bar(midpoints, proportion_change, alpha=0.7, width=(max(proportion_change.index).right / len(midpoints)))
    plt.bar(mid_plus, p_plus, alpha=0.7, width=(max(proportion_change.index).right / len(midpoints)))
    plt.bar(mid_neg, p_neg, alpha=0.7, color='red', width=(max(proportion_change.index).right / len(midpoints)))

    plt.axhline(y=0, color='black', linestyle='-')

    # plt.title(stat + " proportional change between RBERT and DIST Correct predictions")
    plt.xlabel(stat, size=14)
    plt.ylabel("%", size=14)
    plt.yticks([ x for x in range(-100, 125, 25)])
    plt.savefig('images/NCL-Prop')
    plt.show()

In [ ]:
def plot_net_change(stat, verbose=False):
    data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
    data2 = df[(df['relation_label'] == 1)][stat]
    bins = 15

    _, bin1 = pd.cut(data2, bins, retbins=True)
    bins = pd.IntervalIndex.from_breaks(bin1)

    binned_dist = pd.cut(data1, bins)
    bin_vc_dist = binned_dist.value_counts()

    dist_NCP = bin_vc_dist.fillna(0)

    data3 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]

    binned_rbert = pd.cut(data3, bins)
    bin_vc_rbert = binned_rbert.value_counts()

    rbert_NCP = (bin_vc_rbert).fillna(0)

    change = rbert_NCP - dist_NCP

    midpoints = [(bin1[x] + bin1[x+1]) / 2 for x in range(0, len(bin1) -1 , 1)]

    p_plus = [x for x in change.values if x >= 0]
    p_neg = [x for x in change.values if x < 0]
    mid_plus = [midpoints[x] for x, y in enumerate(change.values) if y >= 0]
    mid_neg = [midpoints[x] for x, y in enumerate(change.values) if y < 0]

    if verbose:
        print(p_plus)
        print(bin1)
        print((bin1[0] + bin1[1]) / 2)
        print(midpoints)
        print(max(change.index).right)
        print(len(midpoints))
        print(len(change))
        print(len(p_plus))
        print(len(mid_plus))
        print(len(p_neg))
        print(len(mid_neg))

    # plt.bar(midpoints, proportion_change, alpha=0.7, width=(max(proportion_change.index).right / len(midpoints)))
    plt.bar(mid_plus, p_plus, alpha=0.7, width=(max(change.index).right / len(midpoints)))
    plt.bar(mid_neg, p_neg, alpha=0.7, color='red', width=(max(change.index).right / len(midpoints)))

    plt.axhline(y=0, color='black', linestyle='-')

    # plt.title(stat + " net change between RBERT and DIST Correct predictions")
    plt.xlabel(stat, size=14)
    plt.ylabel("Frequency", size=14)
    # plt.yticks([ x / 100 for x in range(-100, 125, 25)])
    plt.show()
    
# plot_proportional_change('Non-component %')
# plot_change('Document Length')
# plot_change('Non-component Length')
# plot_proportional_change('Bridge Length')
# plot_net_change('Bridge Length')
# plot_proportional_change('Component Distance')
# plot_net_change('Component Distance')
# plot_proportional_change('Document Length')
# plot_net_change('Document Length')
# plot_proportional_change('Non-component %')
# plot_net_change('Non-component %')
# plot_proportional_change_d1('Non-component %')

plot_proportional_change('Non-component Length')
plot_net_change('Non-component Length')
plot_proportional_change_d1('Non-component Length')

In [ ]:
plot_proportional_change('Document Length')
plot_net_change('Document Length')
plot_proportional_change_d1('Document Length')

In [ ]:
plot_proportional_change('Outer Length')
plot_net_change('Outer Length')
plot_proportional_change_d1('Outer Length')

In [ ]:
stat = 'Non-component %'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST \n Non-component / doc length - Reason: Correct predictions against all examples'
print(bins)
plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component token %", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/DIST-NCOT')

In [ ]:
stat = 'Document Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST \n Document length - Reason: Correct predictions against all examples'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="All reason examples")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Correctly predicted reason examples", color='purple')

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Document Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/DIST-DL')

In [ ]:
stat = 'Bridge Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST vs Bridge length - Reason: Correct predictions'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, label="DIST Correct", color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="RBERT Correct")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Both Correct", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Bridge Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERTvDIST-BL')

In [ ]:
stat = 'Non-component Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST vs Non-component length - Reason: Correct predictions'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, label="DIST Correct", color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="RBERT Correct")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Both Correct", color='purple')
# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Non-component length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERTvDIST-NCL')

In [ ]:
stat = 'Document Length'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST vs Document length - Reason: Correct predictions'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, label="DIST Correct", color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="RBERT Correct")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Both Correct", color='purple')

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Document Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')
plt.savefig('images/RBERTvDIST-DL')

In [ ]:
stat = 'target_len'
bins = 15
data1 = df[(df['relation_label'] == 1) & (df['dist_relation_pred'] == 1)][stat]
data2 = df[(df['relation_label'] == 1) & (df['rbert_relation_pred'] == 1)][stat]
bins = np.histogram(np.hstack((data1, data2)), bins=bins)[1]
title = 'DIST vs Document length - Reason: Correct predictions'

plt.figure(figsize=(8,6))
plt.hist(data1, bins=bins, alpha=0.5, label="DIST Correct", color='red')
plt.hist(data2, bins=bins, alpha=0.5, label="RBERT Correct")
plt.hist(data2[0:0], bins=bins, alpha=0.45, label="Both Correct", color='purple')

# plt.hist(df[df['relation_label'] == 1][stat], bins=bins, alpha=0.5, label="Reason")
plt.xlabel("Document Length", size=14)
plt.ylabel("Frequency", size=14)
# plt.title(title)
plt.legend(loc='upper right')

In [ ]:
len(data1)

In [ ]:
# Common correct preds
rsn_dist_c_rbert_c = rsn[(rsn['dist_relation_pred'] == 1) & (rsn['rbert_relation_pred'] == 1)]

plt.hist(rsn_dist_c_rbert_c['Bridge Length'], bins=25)

rsn_dist_c_rbert_c_mean = process_means(rsn_dist_c_rbert_c, 'Both Correct')

rsn_dist_c_rbert_c_mean

In [ ]:
# Common incorrect preds
rsn_dist_f_rbert_f = rsn[(rsn['dist_relation_pred'] != 1) & (rsn['rbert_relation_pred'] != 1)]
# rsn_dist_f_rbert_f_mean = pd.Series(rsn_dist_f_rbert_f.mean()[:], name='Both Incorrect')
# rsn_dist_f_rbert_f_mean['count'] = len(rsn_dist_f_rbert_f)
rsn_dist_f_rbert_f_mean = process_means(rsn_dist_f_rbert_f, 'Both Incorrect')

rsn_dist_f_rbert_c = rsn[(rsn['dist_relation_pred'] != 1) & (rsn['rbert_relation_pred'] == 1)]
# rsn_dist_f_rbert_c_mean = pd.Series(rsn_dist_f_rbert_c.mean()[:], name='RBERT not Dist')
# rsn_dist_f_rbert_c_mean['count'] = len(rsn_dist_f_rbert_c)
rsn_dist_f_rbert_c_mean = process_means(rsn_dist_f_rbert_c, 'RBERT not Dist')

rsn_dist_c_rbert_f = rsn[(rsn['dist_relation_pred'] == 1) & (rsn['rbert_relation_pred'] != 1)]
# rsn_dist_c_rbert_f_mean = pd.Series(rsn_dist_c_rbert_f.mean()[:], name='Dist not RBERT')
# rsn_dist_c_rbert_f_mean['count'] = len(rsn_dist_c_rbert_f)
rsn_dist_c_rbert_f_mean = process_means(rsn_dist_c_rbert_f, 'Dist not RBERT')

means2 = pd.DataFrame(rsn_mean)
means2 = means2.join(rsn_dist_c_rbert_c_mean)
means2 = means2.join(rsn_dist_f_rbert_f_mean)
means2 = means2.join(rsn_dist_f_rbert_c_mean)
means2 = means2.join(rsn_dist_c_rbert_f_mean)
means2 = means2.round(2)

means2


In [ ]:
rsn_dist_c_rbert_c

In [ ]:
for idx in rsn_dist_c_rbert_c.index:
    print('Doc ID: {}'.format(text_info[idx]['document_id']))
    print(text_info[idx]['text'])
    print(text_info[idx]['tokens'])
    print('-------------------------')
    print()

In [ ]:
for idx in rsn_dist_f_rbert_f.index:
    print('Doc ID: {}'.format(text_info[idx]['document_id']))
    print(text_info[idx]['text'])
    print(text_info[idx]['tokens'])
    print('-------------------------')
    print()

In [ ]:
both_correct_docs = set(rsn_dist_c_rbert_c['document_id'].values)
both_incorrect_docs = set(rsn_dist_f_rbert_f['document_id'].values)

both_correct_docs.intersection(both_incorrect_docs)

In [ ]:
both_correct_docs.difference(both_incorrect_docs)

In [ ]:
rbert_correct = set(rbert_rsn_c['document_id'].values)
dist_correct = set(dist_rsn_c['document_id'].values)

bi = both_incorrect_docs.copy()
bi.difference_update(rbert_correct)
bi.difference_update(dist_correct)
bi

In [ ]:
bi

In [ ]:
count = 0
for text in text_info:
    if text['document_id'] in bi and text['relation_label'] == 1:
        print(text['text'])
        print(text['tokens'])
        print('-------------------------')
        print()
        count += 1

In [ ]:
adj_200 = rsn[(rsn['Component Distance']) == 1
                                      & (rsn['Non-component Length'] >= 183.6)
                                      & (rsn['Non-component Length'] <= 213.86667)
                                      & (rsn['relation_label'] != rsn['rbert_relation_pred'])]
process_means(adj_200, 'adj200')
adj_200

In [ ]:
for doc_id in dist_long_correct_ids:
    print(rbert_rsn_c[rbert_rsn_c['document_id'] == doc_id].index)

In [ ]:
rbert_rsn_c.iloc[3138]

In [ ]:
count = 0
for text in text_info:
    if text['document_id'] in bi and text['relation_label'] == 1:
        print(text['text'])
        print(text['tokens'])
        print('-------------------------')
        print()
        count += 1

In [ ]:
print(count)

In [ ]:
text_info['document_id' == 2]

In [ ]:
incorrect = set(df[df['relation_label'] != df['rbert_relation_pred']]['document_id'])
correct = set(df[df['relation_label'] == df['rbert_relation_pred']]['document_id'])

incorrect.difference(correct)

In [ ]:
rbert_rsn_c.sort_values('Non-component Length')

In [ ]:
rsn[rsn['document_id'] == 8]

In [ ]:
text_info[7531]['text']

In [ ]:
text_info[144]['text']

In [ ]:
text_info[158]['text']

In [ ]:
text_info[160]['text']

In [ ]:
text_info[8238]['text']

In [ ]:
text_info[4771]['text']

In [ ]:
text_info[4787]['text']

In [ ]:
text_info[4922]['text']